#### Import libraries


In [1]:
if __name__ == '__main__':
    from nltk.corpus import stopwords
    import csv
    import spacy
    import os
    import pandas as pd
    import time
    import pickle
    import nltk
    import numpy as np
    from sklearn.preprocessing import MinMaxScaler
    #import matplotlib as plt
    from matplotlib import pyplot as plt

#Check execution time
start_time = time.time()

#### Approach
- ###### 1st part all reviews -> pre-processing -> dictionary (existing) -> word : dictionary ->score101
- ###### 2nd part topics sentiment
- ###### unpickling file #Two pickle for two kinds of O/P from LDA

In [2]:
echo='18_topics'
if (echo == '18_topics'):
    pickle_in101 = open("dataOCM/forTSNE_Data_Cleaned_07092020_113759.df", "rb")   #"C:\\Users\\Admin\\PycharmProjects\\CrossCulturalSentimentAnalysis\\forTSNE_Data_Cleaned_07092020_113759.df", "rb")
    labelPlot = ['bad companies',
    'job pressure',
    'work atmosphere - interesting tasks',
    'cohesion among colleagues',
    'treatment of colleagues',
    'equal rights',
    'work life balance',
    'treatment of leader',
    'communication in the organization',
    'flexible work hours',
    'circumstances at work - overtime',
    'age diversity',
    'vacation',
    'work from home',
    'good management',
    'work hours',
    'work atmosphere',
    'work atmosphere for new colleagues']
    print("Inside if")
else:
    pickle_in101 = open("C:\\Users\\Admin\\Downloads\\LDA_04_Fitting_fittedData_08092020_083631.data", "rb")
    labelPlot = ['relationships with colleagues and bosses',
    'satisfaction with work and colleagues',
    'work hours',
    'great education',
    'gender diversity',
    'cultural diversity',
    'overtime, job pressure',
    'work life balance',
    'women empowerment',
    'appreciation of experience',
    'age diversity',
    'cohesion among colleagues',
    'cohesion among new colleagues',
    'fun environment',
    'young team',
    'communication in the organisation',
    'work from home',
    'bad salary',
    'tasks at work',
    'female leaders']

Inside if


In [3]:

sample_data = pickle.load(pickle_in101)
unpickled_df = pd.DataFrame(sample_data)

new_columns = unpickled_df.columns
unpickled_df.columns = new_columns
print('unpickled_df.columns', unpickled_df.columns)

noOfTopics = 0

for name in new_columns:
    if 'topic' in name:
        noOfTopics +=1

for i in range(noOfTopics):
        if (i in []):                                         #01
            continue
        else:
            unpickled_df['topic' + str(i) + '_max'] = unpickled_df['topic' + str(i)]

#unpickled_df = unpickled_df[1:unpickled_df.size]
seriesLen = unpickled_df['RvComment'].size
print('size of dataframe is :', seriesLen)
reviews = []
tokenizer = nltk.RegexpTokenizer(r"\w+")

print(unpickled_df.head())
#print('seriesLen : ', seriesLen)
#print(unpickled_df['RvComment'])
for i in range(1, seriesLen):
    lstRev = []
    lstRev.append(unpickled_df['RvComment'][i])
    reviews.append(lstRev)

# Fetching pos neg word list
pnFile='dataOCM/Pos_NegWordlist.txt'   #'Pos_NegWordlist.txt'
posNegWordList = list(csv.reader(open(pnFile, encoding='utf-8'), delimiter=','))  # CSV file to 2 dimensional list of string
df3=pd.DataFrame(posNegWordList)

#convert to lower

dfPosNegtemp = df3.apply(lambda x: x.astype(str).str.lower())

dfPosNeg_final = dfPosNegtemp[[1, 2] + [c for c in dfPosNegtemp if c not in [1, 2]]]

#Creating a list out of the entire data frame : dfPosNeg_final
#Create an empty list

Row_list=[]

#renaming the headers

dfPosNeg_final.rename(columns={1: "POS", 2: "Score101"}, inplace= True)

dfPosNeg_final.set_index("Score101", inplace=True)

# Get bool dataframe with True at positions where the given value exists


unpickled_df.columns Index(['Org', 'OrgSector', 'RverMonthYear', 'RverReviewer', 'RverPosition',
       'RverLoc', 'RverRecom', 'RvReviewAbout', 'RvScore', 'RvComment',
       'RvScoreWorkAtmosphere', 'RvScoreCohesionAmongColleagues',
       'RvScoreEqualRights', 'RvScoreDealingWithOlderColleagues',
       'RvScoreEnvironmentalSocialAwareness', 'Corona1', 'Corona2', 'Corona3',
       'RvScoreWorkLifeBalance', 'topic0', 'topic1', 'topic2', 'topic3',
       'topic4', 'topic5', 'topic6', 'topic7', 'topic8', 'topic9', 'topic10',
       'topic11', 'topic12', 'topic13', 'topic14', 'topic15', 'topic16',
       'topic17', 'forTSNEYear', 'forTSNECity'],
      dtype='object', name=0)
size of dataframe is : 20997
0                           Org   OrgSector RverMonthYear RverReviewer  \
1  Instone Real Estate Group AG  Immobilien      May 2020            1   
2  Instone Real Estate Group AG  Immobilien      May 2020            2   
3  Instone Real Estate Group AG  Immobilien      May 2020         

In [4]:
#nlpDe = spacy.load('de_core_news_sm')
stop_words_de = stopwords.words('german')

In [5]:

# function: tokenize, lower, stopword removal, and score calculation
def sentimentCalculator(strReview):
    listReviewTokens = tokenizer.tokenize(strReview)
    reviewScore = 0
    doc_out = []
    for doc in listReviewTokens:
        doc = doc.lower()
        if doc not in stop_words_de:  # remove stopwords
            word=[doc]
            doc_out = doc_out + word

    result = dfPosNeg_final.isin(doc_out)
    seriesObj = result.any()
    columnNames = list(seriesObj[seriesObj == True].index)
        # Iterate over list of columns and fetch the rows indexes where value exists
    for col in columnNames:
        rows = list(result[col][result[col] == True].index)
        for row in rows:
                # print('row ====', row, type(row))
            integerRow = float(row)
            reviewScore = reviewScore + integerRow
    return reviewScore

unpickled_df['score101'] = unpickled_df['RvComment'].apply(sentimentCalculator)
print(unpickled_df)

0                               Org                      OrgSector  \
1      Instone Real Estate Group AG                     Immobilien   
2      Instone Real Estate Group AG                     Immobilien   
3      Instone Real Estate Group AG                     Immobilien   
4      Instone Real Estate Group AG                     Immobilien   
5      Instone Real Estate Group AG                     Immobilien   
...                             ...                            ...   
20993        KWS SAAT SE & Co. KGaA  Nahrungsmittel/Landwirtschaft   
20994        KWS SAAT SE & Co. KGaA  Nahrungsmittel/Landwirtschaft   
20995        KWS SAAT SE & Co. KGaA  Nahrungsmittel/Landwirtschaft   
20996        KWS SAAT SE & Co. KGaA  Nahrungsmittel/Landwirtschaft   
20997        KWS SAAT SE & Co. KGaA  Nahrungsmittel/Landwirtschaft   

0      RverMonthYear RverReviewer                       RverPosition  \
1           May 2020            1         Führungskraft / Management   
2           May

##### Topic sentiment calculation

In [6]:
for index, row in unpickled_df.iterrows():
    rowMax = max(row[['topic' + str(c) for c in range(noOfTopics) if c not in [12,20,22]]])
    for j in range(noOfTopics):
        if (j in []):                                                 #02
            continue
        elif row['topic' + str(j) + '_max'] == rowMax:
            # row['topic' + str(j) + '_max'] = 1
            unpickled_df.at[index, 'topic' + str(j) + '_max'] = 1
        else:
            # row['topic' + str(j) + '_max'] = 0
            unpickled_df.at[index, 'topic' + str(j) + '_max'] = 0

topicVsSentiment = []
for t in range(noOfTopics):
    # if (t in [12, 20, 22]):                                                      #03
    #     continue
    topicVsSentiment_temp = []
    # default value
    topicVsSentiment_temp.append('topic' + str(t))
    topicVsSentiment_temp.append(0)
    topicVsSentiment_temp.append(0)
    topicVsSentiment_temp.append('NaN')

    topicVsSentiment.append(topicVsSentiment_temp)
    # topicVsSentiment['topic' + str(t)] = [0,'NaN']
print(topicVsSentiment)
for i in range(noOfTopics):
    if (i in []):                                                       #04
        continue
    unpickled_df['topicXmaxboolXcss' + str(i)] = unpickled_df['topic' + str(i)]*unpickled_df['topic' + str(i) + '_max']*unpickled_df['score101']
    print('topicVsSentiment[i][1] :',topicVsSentiment[i][1])
    topicVsSentiment[i][1] = sum(unpickled_df['topicXmaxboolXcss' + str(i)])


dftopicXSentiment = pd.DataFrame(topicVsSentiment)

# insert column with sentiment type
conditions = [(dftopicXSentiment[1] > 0), (dftopicXSentiment[1] < 0), (dftopicXSentiment[1] ==0)]
choices = ['Positive', 'Negative', 'Neutral']
dftopicXSentiment[3] = np.select(conditions, choices, default='NaN')

[['topic0', 0, 0, 'NaN'], ['topic1', 0, 0, 'NaN'], ['topic2', 0, 0, 'NaN'], ['topic3', 0, 0, 'NaN'], ['topic4', 0, 0, 'NaN'], ['topic5', 0, 0, 'NaN'], ['topic6', 0, 0, 'NaN'], ['topic7', 0, 0, 'NaN'], ['topic8', 0, 0, 'NaN'], ['topic9', 0, 0, 'NaN'], ['topic10', 0, 0, 'NaN'], ['topic11', 0, 0, 'NaN'], ['topic12', 0, 0, 'NaN'], ['topic13', 0, 0, 'NaN'], ['topic14', 0, 0, 'NaN'], ['topic15', 0, 0, 'NaN'], ['topic16', 0, 0, 'NaN'], ['topic17', 0, 0, 'NaN']]
topicVsSentiment[i][1] : 0
topicVsSentiment[i][1] : 0
topicVsSentiment[i][1] : 0
topicVsSentiment[i][1] : 0
topicVsSentiment[i][1] : 0
topicVsSentiment[i][1] : 0
topicVsSentiment[i][1] : 0
topicVsSentiment[i][1] : 0
topicVsSentiment[i][1] : 0
topicVsSentiment[i][1] : 0
topicVsSentiment[i][1] : 0
topicVsSentiment[i][1] : 0
topicVsSentiment[i][1] : 0
topicVsSentiment[i][1] : 0
topicVsSentiment[i][1] : 0
topicVsSentiment[i][1] : 0
topicVsSentiment[i][1] : 0
topicVsSentiment[i][1] : 0


In [7]:
#find max and min
max_pos = dftopicXSentiment[1].max()
max_neg = dftopicXSentiment[1].min()

#print("---------------max",max_pos, max_neg)
#print(dftopicXSentiment[1])

#type conversion
dftopicXSentiment[1] = dftopicXSentiment[1].astype(float)
dftopicXSentiment[2] = dftopicXSentiment[2].astype(float)
#print(dftopicXSentiment.dtypes, "---check type")

#iterate through rows and divide the values, positive numbers with max_pos, and negative numbers with max_neg
#METHOD1
for index, row in dftopicXSentiment.iterrows():
    if (row[1] >0) :
        # unpickled_df.at[index, 'topic' + str(j) + '_max'] = 1
        dftopicXSentiment.at[index, [2]] = row[1]/max_pos
    elif (row[1] < 0): #'Negative'):
        dftopicXSentiment.at[index, [2]] = -row[1]/max_neg
    else:
        dftopicXSentiment.at[index, [2]] = 0
dftopicXSentiment[4]=dftopicXSentiment[1]


In [8]:
unpickled_df.head()

,Org,OrgSector,RverMonthYear,RverReviewer,RverPosition,RverLoc,RverRecom,RvReviewAbout,RvScore,RvComment,...,topicXmaxboolXcss8,topicXmaxboolXcss9,topicXmaxboolXcss10,topicXmaxboolXcss11,topicXmaxboolXcss12,topicXmaxboolXcss13,topicXmaxboolXcss14,topicXmaxboolXcss15,topicXmaxboolXcss16,topicXmaxboolXcss17
1,Instone Real Estate Group AG,Immobilien,May 2020,1,Führungskraft / Management,Hat zum Zeitpunkt der Bewertung im Bereich Fin...,Empfohlen,Overall,4.9,Top Arbeitgeber mit vielen Benefiz Trans...,...,0,0,0,0,0,0,0.128307,0,0,0
2,Instone Real Estate Group AG,Immobilien,May 2020,2,Angestellte/r oder Arbeiter/in,Hat zum Zeitpunkt der Bewertung im Bereich Rec...,Empfohlen,Overall,4.9,Vorbildlicher Arbeitgeber Nur die Mülltre...,...,-0,-0,-0,-0,-0,-0,-0.085525,-0,-0,-0
3,Instone Real Estate Group AG,Immobilien,May 2020,3,Angestellte/r oder Arbeiter/in,Hat zum Zeitpunkt der Bewertung im Bereich Adm...,Empfohlen,Overall,4.5,Wenn Transparenz und Wertschätzung aufeinande...,...,0,0,0,0,0,0,0,0,0,0
4,Instone Real Estate Group AG,Immobilien,October 2019,4,Ex-Angestellte/r oder Arbeiter/in,Hat im Bereich Personal / Aus- und Weiterbildu...,Nicht empfohlen,Overall,3.2,Anspruch und Wirklichkeit klaffen auseinander...,...,0,0,0,0.0711214,0,0,0,0,0,0
5,Instone Real Estate Group AG,Immobilien,October 2019,5,Führungskraft / Management,Hat zum Zeitpunkt der Bewertung bei Instone Re...,Empfohlen,Overall,4.8,Toller Arbeitgeber Tolle Truppe Auch bei lä...,...,0,0,0,0,0,0,0,0,0,0


In [9]:

unpickled_df.to_csv("sentimentReviews_113759.csv")
#dftopicXSentiment.to_csv("dftopicXSentiment_DATASET1.csv")
unpickled_df.to_pickle("sentiment_score_reviews_113759.data")

print("--- Total Execution Time : %s seconds ---" % (time.time() - start_time))

--- Total Execution Time : 288.3107817173004 seconds ---
